# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.csv')
sales.head()

Finished parsing file /home/sadat/Coursera/ML regression/Week 5/kc_house_data.csv

Parsing completed. Parsed 100 lines in 0.091314 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,int,float,int,int,float,int,int,int,int,int,int,int,int,int,float,float,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/sadat/Coursera/ML regression/Week 5/kc_house_data.csv

Parsing completed. Parsed 21613 lines in 0.083153 secs.

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view
7129300520,20141013T000000,221900.0,3,1.0,1180,5650,1.0,0,0
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0
5631500400,20150225T000000,180000.0,2,1.0,770,10000,1.0,0,0
2487200875,20141209T000000,604000.0,4,3.0,1960,5000,1.0,0,0
1954400510,20150218T000000,510000.0,3,2.0,1680,8080,1.0,0,0
7237550310,20140512T000000,1225000.0,4,4.5,5420,101930,1.0,0,0
1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0
2008000270,20150115T000000,291850.0,3,1.5,1060,9711,1.0,0,0
2414600126,20150415T000000,229500.0,3,1.0,1780,7470,1.0,0,0
3793500160,20150312T000000,323000.0,3,2.5,1890,6560,2.0,0,0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [8]:
import numpy as np
import pprint 
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [9]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [10]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.059060     | 6869667.894269     | 385998.458664 |

| 6         | 7        | 0.000002  | 1.147430     | 6847177.772191     | 380824.456039 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [18]:
print model_all["coefficients"][model_all["coefficients"]["value"] > 0]

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 274873.055652 |  None  |
|    bathrooms     |  None | 8468.53105208 |  None  |
|   sqft_living    |  None | 24.4207211037 |  None  |
| sqft_living_sqrt |  None | 350.060551005 |  None  |
|      grade       |  None | 842.068028819 |  None  |
|    sqft_above    |  None | 20.0247226436 |  None  |
+------------------+-------+---------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [19]:
rss_set = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty)
    prediction = model.predict(validation)
    error = validation['price'] - prediction
    rss = (error * error).sum()
    rss_set[l1_penalty] = rss
print min(rss_set.items(), key=lambda x: x[1]) 

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.756133     | 5659149.969577     | 285408.885360 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

(10.0, 625766294657736.8)


In [20]:
modeltest = graphlab.linear_regression.create(testing, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=10)
prediction = modeltest.predict(validation)
error = validation['price'] - prediction
rss = (error * error).sum()
print rss

Linear regression:

--------------------------------------------------------

Number of examples          : 2217

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000018  | 0.397419     | 2480795.833850     | 274447.082100 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

6.31133653187e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [23]:
print modeltest["coefficients"][modeltest["coefficients"]["value"] > 0]
len(modeltest["coefficients"][modeltest["coefficients"]["value"] > 0])

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 15938.3496994 |  None  |
|     bedrooms     |  None | 7927.96167002 |  None  |
| bedrooms_square  |  None | 2811.65523092 |  None  |
|    bathrooms     |  None | 25090.8110442 |  None  |
|   sqft_living    |  None | 39.6894149442 |  None  |
| sqft_living_sqrt |  None | 1115.10435061 |  None  |
|  sqft_lot_sqrt   |  None | 126.735051408 |  None  |
|      floors      |  None | 19806.9707703 |  None  |
|  floors_square   |  None | 12646.6383093 |  None  |
|    waterfront    |  None | 413704.308549 |  None  |
+------------------+-------+---------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


17

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [24]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [25]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [28]:
coef_set = {}
for l1_penalty in np.logspace(8, 10, num=20):
    modelnew = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty)
    coef_set[l1_penalty] = modelnew["coefficients"]["value"].nnz()

print min(coef_set.items(), key=lambda x: x[1])
print max(coef_set.items(), key=lambda x: x[1]) 

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000012  | 0.678628     | 7187431.217483     | 380705.226348 |

+-----------+----------+-----------+--------------+--------------------+---------------+

(10000000000.0, 1)
(100000000.0, 18)


TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [33]:


pprint.pprint(coef_set)
l1_penalty_min = 2976351441.6313128
l1_penalty_max = 3792690190.7322536


{100000000.0: 18,
 127427498.57031322: 18,
 162377673.91887242: 18,
 206913808.111479: 18,
 263665089.87303555: 17,
 335981828.6283788: 17,
 428133239.8719396: 17,
 545559478.1168514: 17,
 695192796.1775591: 17,
 885866790.4100832: 16,
 1128837891.6846883: 15,
 1438449888.2876658: 15,
 1832980710.8324375: 13,
 2335721469.0901213: 12,
 2976351441.6313133: 10,
 3792690190.7322536: 6,
 4832930238.571753: 5,
 6158482110.6602545: 3,
 7847599703.514623: 1,
 10000000000.0: 1}


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [39]:
rss_newset = {}
for l1_penalty in np.linspace(l1_penalty_min,l1_penalty_max,20):
    model1 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=l1_penalty)
    prediction = model1.predict(validation)
    error = validation['price'] - prediction
    rss = (error * error).sum()
    rss_newset[l1_penalty] = rss,model1["coefficients"]["value"].nnz()
pprint.pprint(rss_newset)

Linear regression:

--------------------------------------------------------

Number of examples          : 9761

Number of features          : 17

Number of unpacked features : 17

...

| 6         | 7        | 0.000003  | 0.828406     | 6697293.359582     | 370685.052351 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

{2976351441.631313: (966925686454724.4, 10),
 3019316638.952415: (974019443956344.8, 10),
 3062281836.273517: (981188361593344.4, 10),
 3105247033.5946193: (989328337463313.4, 10),
 3148212230.9157214: (998783205406022.2, 10),
 3191177428.2368236: (1008477160913734.2, 10),
 3234142625.5579257: (1018298774200969.5, 10),
 3277107822.879028: (1028247985606636.8, 10),
 3320073020.20013: (1034616913873238.1, 8),
 3363038217.521232: (1038554740571297.4, 8),
 3406003414.8423343: (1043237239891139.2, 8),
 3448968612.1634364: (1046937489533987.0, 7),
 3491933809.4845386: (1051147625263920.8, 7),
 3534899006.8056407: (1055992734935847.1, 7),
 3577864204.126743: (1060799531527687.6, 7),
 3620829401.447845: (1065707687780826.2, 6),
 3663794598.768947: (1069464333664018.4, 6),
 3706759796.0900493: (1073504547508641.2, 6),
 3749724993.4111514: (1077632773453153.2, 6),
 3792690190.7322536: (1081867590469109.9, 6)}


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [38]:


print model1["coefficients"][model1["coefficients"]["value"] > 0]
len(model1["coefficients"][model1["coefficients"]["value"] > 0])

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 240309.758568 |  None  |
|    bathrooms     |  None | 13840.6401156 |  None  |
|   sqft_living    |  None | 30.5583595091 |  None  |
| sqft_living_sqrt |  None | 592.199472839 |  None  |
|      grade       |  None | 2265.12053218 |  None  |
|    sqft_above    |  None | 27.4878733975 |  None  |
+------------------+-------+---------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use sf.materialize() to force materialization.


6